# Example client application
Making calls to the PyUniRPC server via ZMQ

In [ ]:
import zmq
import pyunirpc as prpc
import numpy as np
import json

### Set up connection

In [ ]:
context = zmq.Context()
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:6789")
print("Connected")

### Prepare RPC call
Numpy arrays are dealt with as a special case, such that data can be sent in binary form (via Base64).

In [ ]:
raw_call = prpc.prepare_call(
    handle = 'my_function',            #This is the name we defined server-side
    uid    = 42,                       #An ID for this request
    args = [17, 'this is a string'],   #Positional arguments
    kwargs = dict(array=np.array([1,2,3])) #Keyword arguments dict
)
print("Raw call dictionary:\n--------------------")
print(raw_call)

### Dispatch call and receive result

In [ ]:
print("Dispatching RPC call...")
socket.send(json.dumps(raw_call).encode()) #The raw call is converted to JSON bytes
print("Receiving result...")
raw_result = json.loads(socket.recv()) #The result is decoded from JSON to a dict
print("Raw result dictionary:\n----------------------")
print(raw_result)


if raw_result['rpc_tag'] == '__RPC_RESULT__':
    result = prpc.decode_vals(raw_result['result']) #Decode the result tuple
    print("\nResult data `(type, value)`:\n----------------------------")
    for r in result: print(type(r), r)
else: #`raw_result['rpc_tag']` will be `__RPC_ERROR__`
    print("\nRPC error:\n----------")
    print(raw_result['exception'])
    print(raw_result['descr'])